In [1]:
import pandas as pd
import os
import gc
import pickle
import numpy as np

from time import sleep

import gspread_dataframe as gs_df

os.chdir("c:/Users/rsilva/gdrive/projects/programming/automation/")

PATH = os.path.join(os.getcwd(), 'files/')

# INITIALIZE DRIVER
from selenium import webdriver
import sapiens
import functions
import sistemas


% reload_ext autoreload
% autoreload 2

CPF = '33831640807'
PASS_1 = "Fer465023@ur_"
PASS_2 = "Fer465023"

In [2]:
!ls {PATH}

Banco_de_Dados_CassaÃ§Ã£o.xlsx
anatel.json
conda-env-file.sh
correspondencias_devolvidas.xlsx
csv
planilha_auxiliar_CADIN.xlsm
planilha_auxiliar_CADIN.xlsx


### Loga Driver no Sapiens

In [6]:
def start_sapiens(cpf, senha, db={}):   
   
    page = functions.init_browser(webdriver.Firefox(), 'rsilva', 'Savorthemom3nts')
    
    page = sapiens.LoginPage(page.driver).login(cpf, senha)
    
    page = sapiens.Sapiens(page.driver, db)
        
    return page

In [13]:
def fix_cpf_cnpj(_id):
    
    if len(_id) in (12, 13, 14):
        
        while len(_id) < 14:
            
            _id = '0' + str(_id)
            
        return _id
            
    elif len(_id) <= 11:
    
        while len(_id) < 11:

            _id = '0' + str(_id)
            
        return _id
    
    return ""

### Cria/Carrega o Data Frame para exportar os dados

In [40]:
database = pd.read_excel(PATH+"planilha_auxiliar_CADIN.xlsx", dtype=str).replace("nan", "")

In [41]:
database.head()

,Fistel,Seq,Processo,Nome,CPF/CNPJ,Serviço,Receita,Data Vencimento,Valor Original,Data Pagamento,Situação
0,02000405029,1,,ACACIO MAURICIO DE OLIVEIRA NETO,1332147887,302 - Radioamador,1329 - TFF,31/03/1988\r,&nbsp;Histórico do Lançamento</b,,Quitado
1,01033388297,1,,ACHILLES FERNANDO CATAPANI ABELAIRA,1173626891,507 - Móvel Aeronáutico,8766 - TFI,17/09/1997\r,&nbsp;Histórico do Lançamento</b,17/09/1997,Quitado
2,02035273560,1,,ADAILTON JOSE ROSA,5343241859,302 - Radioamador,8766 - TFI,18/11/1996\r,&nbsp;Histórico do Lançamento</b,18/11/1996,Quitado
3,02015316230,1,,ADALBERTO SANTOS OLIVEIRA,88513343820,400 - Rádio do Cidadão,1329 - TFF,31/03/1989\r,&nbsp;Histórico do Lançamento</b,,Quitado
4,02000897576,1,,ADANELSON CORREA,51917815891,302 - Radioamador,1329 - TFF,31/03/1988\r,&nbsp;Histórico do Lançamento</b,,Quitado


In [45]:
database.shape

(10176, 11)

In [44]:
database.dropna(inplace=True)

In [7]:
sapiens_page = start_sapiens(CPF, PASS_2)

In [59]:
df = pd.DataFrame(columns=sapiens.KEYS)

In [60]:
counter = 0

for i, row in database.iterrows():
    
    cpf = row["CPF/CNPJ"].replace(".", "").replace('-', "")
    
    cpf = fix_cpf_cnpj(cpf)
    
    if len(cpf) > 11: next
    
    registro = sapiens_page.get_registro(cpf)    
   
    if registro is None:
            
        sapiens_page.pesquisa_dados(cpf)
        
        registro = sapiens_page.get_registro(cpf)
        
        counter += 1
        
    df = df.append(pd.Series(registro), ignore_index=True)
        
    if counter > 100:
        
        df.to_csv("registros_atualizados.csv", index=False)
        
        new = start_sapiens(CPF, PASS_2, sapiens_page.registros) 
        
        sapiens_page.driver.close()
        
        sapiens_page = new
        
        sleep(10)        
                
        counter = 0 
        
        gc.collect()
        
        
else:
    
    df.to_csv("registros_atualizados.csv", index=False)


In [62]:
df.to_excel("registros_atualizados.xlsx", index=False)

In [46]:
df["Endereço_Completo"] = df.apply(lambda row: " ".join([str(row["Logradouro"]), 
                                                         str(row["Número"]), 
                                                         str(row["Complemento"]),
                                                         str(row["Bairro"]), "\n", "CEP:", 
                                                         str(row["Cep"]), 
                                                         str(row["Cidade"])]
                                                       ), axis=1
                                  )

In [65]:
#writer = pd.ExcelWriter("files/planilha_obito.xlsx")
#df.to_excel(writer, "consulta_RF", index=False, encoding='utf-8')
#writer.save()

In [129]:
#gs_df.set_with_dataframe(registros_rf, df, row=1, include_column_header=True)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Nome"]), row=1, col=9, include_column_header=False)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Endereço_Completo"]), row=1, col=12, include_column_header=False)

In [48]:
gs_df.set_with_dataframe(registros, df, row=743, col=14, include_column_header=False)